In [ ]:
!pip install transformers
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import os

In [ ]:
import torch
import os

# Set the API key
os.environ['ACCESS_TOKEN']=""



In [ ]:
from transformers import RagTokenizer, RagTokenForGeneration
import os

api_key = os.getenv('ACCESS_TOKEN')
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq", use_auth_token=api_key)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", use_auth_token=api_key)


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = df.drop(['URL', 'GT'], axis=1)
df.head()

,Question,Option A,Option B,Option C,Option D,Prompt
0,Which of the following mitigations involves pr...,Audit,Execution Prevention,Operating System Configuration,User Account Control,You are given a multiple-choice question (MCQ)...
1,Which data source is recommended for monitorin...,Command,File,Process,User Account,You are given a multiple-choice question (MCQ)...
2,What does mitigation ID M1028 suggest to preve...,Limiting privileges of cloud accounts,Preventing unsigned applications from running,Minimizing applications with setuid or setgid ...,Enforcing the highest UAC level,You are given a multiple-choice question (MCQ)...
3,Which process creation is an indicator of pote...,C:\Windows\System32\services.exe,C:\Windows\System32\cmd.exe,C:\Windows\System32\rundll32.exe,C:\Windows\System32\notepad.exe,You are given a multiple-choice question (MCQ)...
4,"In a Linux environment, what is recommended to...",Monitor Windows Registry Key Modification,Monitor OS API Execution,Monitor file metadata for setuid or setgid bit...,Audit process metadata changes,You are given a multiple-choice question (MCQ)...


In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
def format_input(row):
    question = row['Question']
    options = f"Option A: {row['Option A']}, Option B: {row['Option B']}, Option C: {row['Option C']}, Option D: {row['Option D']}"
    return {"question": question, "options": options}

formatted_inputs = df.head(5).apply(format_input, axis=1)

In [ ]:


def generate_responses(formatted_inputs, batch_size=10):
    results = []
    num_batches = (len(formatted_inputs) + batch_size - 1) // batch_size  # Ensures all items are processed
    for i in tqdm(range(num_batches)):
        batch = formatted_inputs[i*batch_size:(i+1)*batch_size]
        for item in batch:
            question = item['question']
            options = item['options']
            responses = {}
            for option in options.split(", "):
                input_text = f"{question} {option}"
                response = pipe(input_text, max_length=50, num_return_sequences=1)[0]['generated_text']
                responses[option] = response

            # Implement a better scoring mechanism
            # Here we use a dummy scoring function; replace with a more robust approach if possible
            best_answer = max(responses, key=lambda x: len(responses[x]))  # Replace with your own scoring function
            results.append({"question": question, "best_answer": best_answer})

    return results

# Generate responses for the dataset
results = generate_responses(formatted_inputs)
print(results)


  0%|          | 0/1 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pa

[{'question': "Which of the following mitigations involves preventing applications from running that haven't been downloaded from legitimate repositories?", 'best_answer': 'Option A: Audit'}, {'question': 'Which data source is recommended for monitoring commands that may circumvent mechanisms designed to control elevation of privileges?', 'best_answer': 'Option A: Command'}, {'question': 'What does mitigation ID M1028 suggest to prevent privilege escalation exploits on a system?', 'best_answer': 'Option A: Limiting privileges of cloud accounts'}, {'question': 'Which process creation is an indicator of potential SYSTEM privilege escalation according to the detection section?', 'best_answer': 'Option D: C:\\Windows\\System32\\notepad.exe'}, {'question': 'In a Linux environment, what is recommended to monitor for detecting privilege escalation via sudo?', 'best_answer': 'Option A: Monitor Windows Registry Key Modification'}]


In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save to CSV
results_df.to_csv('path_to_save_results.csv', index=False)
